In [13]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import FunctionTransformer

# data = pd.read_csv(r'C:\Users\Saikrishnan.000\Downloads\diagnosis_1.csv', header=None, sep='\t')
data = pd.read_csv('C:\\Users\\Saikrishnan.000\\Downloads\\diagnosis_1.csv')

# Parsing and preprocessing function
def preprocess_data(data):
    processed_data = data.iloc[:, :-2]
    labels = data.iloc[:, -2:]

    # Replace 'no' with 0 and 'yes' with 1
    processed_data.replace({'no': 0, 'yes': 1}, inplace=True)
    labels.replace({'no': 0, 'yes': 1}, inplace=True)

    # Convert temperature values to the correct format
    processed_data.iloc[:, 0] = processed_data.iloc[:, 0].str.replace(',', '.').astype(float)
    
    return processed_data, labels

X, y = preprocess_data(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

print(y_test)


tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 1.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 1.],
        [1., 0.],
        [0., 0.],
        [1., 1.],
        [1., 0.],
        [1., 1.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 1.],
        [1., 0.]])


C:\Users\Saikrishnan.000\AppData\Local\Temp\ipykernel_26760\3610589871.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  processed_data.replace({'no': 0, 'yes': 1}, inplace=True)
C:\Users\Saikrishnan.000\AppData\Local\Temp\ipykernel_26760\3610589871.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.replace({'no': 0, 'yes': 1}, inplace=True)
C:\Users\Saikrishnan.000\AppData\Local\Temp\ipykernel_26760\3610589871.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explici

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(6, 64)  # Assuming 6 features
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)  # First hidden layer to second hidden layer
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 16)  # Second hidden layer to third hidden layer
        self.act3 = nn.ReLU()
        self.fc4 = nn.Linear(16, 2)
        self.sigmoid = nn.Sigmoid()

#         self.fc2 = nn.Linear(64, 2)  # Two outputs: inflammation and nephritis

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Sigmoid for binary classification
        return x

# Instantiate the model, define loss function and optimizer
model = SimpleNN()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [15]:
def train(model, X_train, y_train, optimizer, criterion, epochs=15):
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        # Convert outputs to binary predictions
        predictions = (outputs > 0.5).int()  # Convert probabilities to 0 or 1
        
        # Calculate accuracy for disease 1
        correct_disease1 = (predictions[:, 0] == y_train[:, 0]).float().sum()
        accuracy_disease1 = correct_disease1 / y_train.size(0)
        
        # Calculate accuracy for disease 2
        correct_disease2 = (predictions[:, 1] == y_train[:, 1]).float().sum()
        accuracy_disease2 = correct_disease2 / y_train.size(0)
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Accuracy (Disease 1): {accuracy_disease1.item()}, Accuracy (Disease 2): {accuracy_disease2.item()}')

# Training the model
train(model, X_train, y_train, optimizer, criterion, epochs=15)






Epoch 1/15, Loss: 0.7053012251853943, Accuracy (Disease 1): 0.46315789222717285, Accuracy (Disease 2): 0.43157893419265747
Epoch 11/15, Loss: 0.682537853717804, Accuracy (Disease 1): 0.46315789222717285, Accuracy (Disease 2): 0.43157893419265747


In [16]:
criterion = nn.BCEWithLogitsLoss()
def train_1(model, X_train, y_train, optimizer, criterion, epochs=20, threshold=0.5):
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # Apply sigmoid and threshold to output logits to get binary predictions
        predictions = torch.sigmoid(outputs) > threshold
        # Calculate the number of correct predictions
        correct = (predictions == y_train).float().sum()
        # Calculate accuracy considering all label predictions
        accuracy = correct / (y_train.size(0) * y_train.size(1))
        if epoch % 10 == 0:
          print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Accuracy: {accuracy.item()} Criterion{criterion}')

train_1(model, X_train, y_train, optimizer, criterion, epochs=25)

Epoch 1/25, Loss: 0.726034939289093, Accuracy: 0.4842105209827423 CriterionBCEWithLogitsLoss()
Epoch 11/25, Loss: 0.7202516198158264, Accuracy: 0.4842105209827423 CriterionBCEWithLogitsLoss()
Epoch 21/25, Loss: 0.7130305767059326, Accuracy: 0.4842105209827423 CriterionBCEWithLogitsLoss()


In [17]:
def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        # Convert outputs to binary predictions
        predicted = (outputs.data > 0.5).float()
        # Calculate accuracy for disease 1
        correct_disease1 = (predicted[:, 0] == y_test[:, 0]).float().sum()
        accuracy_disease1 = correct_disease1 / y_test.size(0)
        # Calculate accuracy for disease 2
        correct_disease2 = (predicted[:, 1] == y_test[:, 1]).float().sum()
        accuracy_disease2 = correct_disease2 / y_test.size(0)
        print(f'Accuracy (Disease 1): {accuracy_disease1.item():.4f}, Accuracy (Disease 2): {accuracy_disease2.item():.4f}')

# Evaluate the model
evaluate(model, X_test, y_test)



Accuracy (Disease 1): 0.9167, Accuracy (Disease 2): 0.9583


In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Function to create UI and handle predictions
def acute_inflammation_ui():
    # Text input for temperature
    temp_input = widgets.Text(
        value='37.0',
        description='Temperature:',
        disabled=False
    )

    # Checkboxes for symptoms
    symptoms_labels = [
        'Nausea', 'Lumbar pain', 'Urine pushing',
        'Micturition pains', 'Burning of urethra itch swelling of urethra outlet'
    ]
    symptoms_checkboxes = [widgets.Checkbox(value=False, description=label) for label in symptoms_labels]

    # Button for prediction
    predict_btn = widgets.Button(description="Predict")

    # Output area
    output_area = widgets.Output()

    # Display inputs
    display(temp_input, *symptoms_checkboxes, predict_btn, output_area)

    # Prediction function
    def on_predict_clicked(b):
        with output_area:
            clear_output(wait=True)  # Clear the previous output
            try:
                # Prepare the input data
                temp = float(temp_input.value.replace(",", "."))
                features = [temp] + [1.0 if cb.value else 0.0 for cb in symptoms_checkboxes]
#                 data = scaler.transform([features[:-1]])  # Apply scaling
#                 data_tensor = torch.tensor(data, dtype=torch.float32)
                data_tensor = torch.tensor([features], dtype=torch.float32)

                # Predict
                model.eval()
                with torch.no_grad():
                    prediction = model(data_tensor)
                    prediction = prediction.numpy()[0]
                    inflammation_pred = "Yes" if prediction[0] > 0.5 else "No"
                    nephritis_pred = "Yes" if prediction[1] > 0.5 else "No"

                print(f"Inflammation of urinary bladder: {inflammation_pred}")
                print(f"Nephritis of renal pelvis origin: {nephritis_pred}")
            except Exception as e:
                print(f"Error: {str(e)}")

    # Bind the prediction function to the predict button
    predict_btn.on_click(on_predict_clicked)

# Call the function to display the UI
acute_inflammation_ui()


Text(value='37.0', description='Temperature:')

Checkbox(value=False, description='Nausea')

Checkbox(value=False, description='Lumbar pain')

Checkbox(value=False, description='Urine pushing')

Checkbox(value=False, description='Micturition pains')

Checkbox(value=False, description='Burning of urethra itch swelling of urethra outlet')

Button(description='Predict', style=ButtonStyle())

Output()

<h1> Federated learning actual implementation<h1>

<h3>Simulate Clients</h3>

In [20]:
def simulate_clients(data, labels, n_clients):
    """Simulate n clients by splitting the data and labels."""
    client_data = torch.split(data, int(len(data) / n_clients))
    client_labels = torch.split(labels, int(len(labels) / n_clients))
    return list(zip(client_data, client_labels))

# Simulate 4 hospital clients
client_datasets = simulate_clients(X_train, y_train, 4)


<h3>Train on Clients</h3>

In [21]:
def train_on_client(model, data, labels, criterion, optimizer, epochs=5):
    """Train the model on a client's dataset."""
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    return model.state_dict(), loss.item()


<h3>Aggregate Model Updates</h3>

In [22]:
def aggregate_model_updates(model_updates):
    """Aggregate model updates by averaging the weights."""
    new_state_dict = {}
    for key in model_updates[0].keys():
        new_state_dict[key] = torch.mean(torch.stack([update[key] for update in model_updates]), dim=0)
    return new_state_dict


<h3>Federated Training Loop</h3>


In [23]:
import numpy as np
global_model = SimpleNN()
global_optimizer = optim.Adam(global_model.parameters(), lr=0.001)
criterion = nn.BCELoss()

n_rounds = 5  # Number of federated learning rounds
    
for round_num in range(n_rounds):
    local_model_updates = []
    for client_data, client_labels in client_datasets:
        # Each client gets a fresh copy of the global model
        local_model = SimpleNN()
        local_model.load_state_dict(global_model.state_dict())
        local_optimizer = optim.Adam(local_model.parameters(), lr=0.001)

        # Train on the client's data
        local_update, loss = train_on_client(
            local_model, client_data, client_labels, criterion, local_optimizer
        )
        local_model_updates.append(local_update)

    # Aggregate the updates to form a new global model
    new_global_state_dict = aggregate_model_updates(local_model_updates)
    global_model.load_state_dict(new_global_state_dict)




<h3>Evaluate the Global Model</h3>

In [24]:
evaluate(global_model, X_test, y_test)


Accuracy (Disease 1): 0.8750, Accuracy (Disease 2): 0.9583
